# Squamata Batch Replace ScienceBase XML
A Jupyter Notebook used to download multiple XML Metadata files from ScicenceBase, make a change to this XML file and re-upload.

Uses code examples found in autoSB.py, https://github.com/pbrown-usgs/science-base-automationMT/blob/first-branch/autoSB.py

**Version 1.0**
 - Update child metadata based upon a known directory and file structure

In [1]:
# Load required Libraries
import sys
import os
from lxml import etree
import pandas as pd
import numpy as np
import json
import sys
import sciencebasepy as pysb
import glob

#Create Function that allows one to implement markdown display when printing
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [2]:
#Set Data Paths - perhaps we'll get a user form to do this some day?
dataPath = r"C:\CurrentWork\DataReleases\Silverton2019\Data" #The 'r' signifies a string literal. Use for paths.

In [3]:
#List The Directories
dataDirList = next(os.walk(dataPath))[1]
dataDirList

['AMTAG01',
 'AMTAG02',
 'AMTAG03',
 'AMTAG04',
 'AMTAG05',
 'AMTAG06',
 'AMTCC03',
 'AMTEG06',
 'AMTEG07',
 'AMTEG08',
 'AMTKM01',
 'AMTKM02',
 'AMTKM03',
 'AMTKM04',
 'AMTKM05',
 'AMTMB06',
 'AMTMB07',
 'AMTMB08',
 'AMTMB09',
 'AMTMC01',
 'AMTMC02',
 'AMTSF01',
 'AMTSF02',
 'AMTSF03',
 'AMTSF04',
 'AMTSF05',
 'AMTSF06']

In [4]:
#Now List the xml files
#Look for EDI file to load
dataStationPath = dataPath + '\\' + dataDirList[0]
xmlList = glob.glob(os.path.join(dataStationPath, '*.xml'),  recursive=True)
xmlList
#xmlPath = xmlList[0]
#ediList
#print ('XML File Path:\n' + xmlPath)
#ediPathArray = ediPath.split('\\')
#ediFile = str(ediPathArray[len(ediPathArray)-1])
#print ('EDI File:\n' + ediFile)
#EdiFileName = ediFile
#ediPathArray

['C:\\CurrentWork\\DataReleases\\Silverton2019\\Data\\AMTAG01\\USA-Colorado-Silverton_Caldera-2019-AMTAG01.xml']

In [5]:
#Based on the above let's make a list of file names to overright with:
#Create file pointer array:
xmlFileAndPathArray = []
xmlFileNameArray = []
i=0
for item in dataDirList:
 xmlFileAndPathArray.append(dataPath + '\\' + dataDirList[i] + '\\' \
    + 'USA-Colorado-Silverton_Caldera-2018-' + dataDirList[i] + '.xml')
 xmlFileNameArray.append('USA-Colorado-Silverton_Caldera-2018-' + dataDirList[i] + '.xml')
 i= i+1

#Print array
for i in range(len(xmlFileAndPathArray)): 
 print (xmlFileAndPathArray[i])
 print (xmlFileNameArray[i])


C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG01\USA-Colorado-Silverton_Caldera-2018-AMTAG01.xml
USA-Colorado-Silverton_Caldera-2018-AMTAG01.xml
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG02\USA-Colorado-Silverton_Caldera-2018-AMTAG02.xml
USA-Colorado-Silverton_Caldera-2018-AMTAG02.xml
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG03\USA-Colorado-Silverton_Caldera-2018-AMTAG03.xml
USA-Colorado-Silverton_Caldera-2018-AMTAG03.xml
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG04\USA-Colorado-Silverton_Caldera-2018-AMTAG04.xml
USA-Colorado-Silverton_Caldera-2018-AMTAG04.xml
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG05\USA-Colorado-Silverton_Caldera-2018-AMTAG05.xml
USA-Colorado-Silverton_Caldera-2018-AMTAG05.xml
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG06\USA-Colorado-Silverton_Caldera-2018-AMTAG06.xml
USA-Colorado-Silverton_Caldera-2018-AMTAG06.xml
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTCC03\USA-Colorado-Silverton_Caldera-2018-AMT

In [6]:
#Login to ScienceBase
sb = pysb.SbSession()
username = input("Username:  ")
sb.loginc(str(username))

#Check to see if login is successful
sb.is_logged_in()

#Get the ScienceBase Login session info
sb.get_session_info()

Username:  pbrown@usgs.gov
········


{'fullDisplayName': 'Philip J Brown II [pbrown@usgs.gov]',
 'isLoggedIn': True,
 'jossoSessionId': 'CE1D4A6FEE93D5DE5D4A8F936BCFDCE9',
 'displayName': 'Philip J Brown II',
 'email': 'pbrown@usgs.gov',
 'username': 'pbrown@usgs.gov'}

In [7]:
#Now lets check all the child items.

catNum = '5c5c72b9e4b070828902cb07' #Make sure one has the correct catalog number
item_json = sb.get_item(catNum)

#print ("Public Item: \n\t" + str(item_json))
print(json.dumps(item_json, indent=2, sort_keys=True))

{
  "body": "This data release includes audiomagnetotelluric (AMT) sounding data collected in July 2018 in the Silverton Caldera complex, Colorado, in the Southern Rocky Mountain Volcanic Field, by the U.S. Geological Survey (USGS). Along with geologic mapping, airborne magnetics, airborne electromagnetics, and magnetotellurics, the USGS collected AMT data at 24 sites along four profiles ranging from 3 to 6 kilometers in length across the north-western structural margin of the Silverton caldera in Mineral Basin (MB01-MB05), across the south-eastern margin of the caldera along Cunningham Creek (CC01-CC05), within the caldera in Eureka Graben (EG01-EG05), and within the caldera along upper Cement Creek near the Gold King mine (GK01-GK09). Only the station locations are provided here. The sounding data and documentation are provided in ndividual links.",
  "browseCategories": [
    "Data"
  ],
  "browseTypes": [
    "Shapefile",
    "Downloadable",
    "Map Service",
    "OGC WMS Layer",


In [8]:
#Now let's get all the children catalogs
children = sb.get_child_ids(catNum)
#Print array
for i in range(len(children)): 
 print (children[i])


5ce32ef6e4b0f7ebfdfb8f2d
5ce42c04e4b0654fe0bce479
5ce42946e4b0654fe0bce448
5ce42b7ae4b0654fe0bce46f
5ce42d0ae4b0654fe0bce498
5ce42bb5e4b0654fe0bce475
5ce42d87e4b0654fe0bce4a3
5ce42cd7e4b0654fe0bce48f
5ce42c71e4b0654fe0bce484
5ce42dc1e4b0654fe0bce4a7
5ce32e6ae4b0f7ebfdfb8f26
5ce42873e4b0654fe0bce43d
5ce32ebee4b0f7ebfdfb8f2b
5ce32e9be4b0f7ebfdfb8f29
5ce427fde4b0654fe0bce438
5ce42ca5e4b0654fe0bce48a
5ce42d46e4b0654fe0bce49b
5ce4252be4b0654fe0bce409
5ce42994e4b0654fe0bce44c
5ce32f15e4b0f7ebfdfb8f2f
5ce429dde4b0654fe0bce451
5ce428aae4b0654fe0bce440
5ce428dfe4b0654fe0bce443
5ce42b3ee4b0654fe0bce46b


In [9]:
#First we get the json item for each catalog number.  
#We need to check to make sure the order of the catalog items is the same as the files we want to upload
#Let's make a list of catalog numbers and 

arrayName = []
for i in range(len(children)):
 childItem_json = sb.get_item(children[i])
 for item in childItem_json['files']:
  fileEXT = item['name'].split('.', 1)[-1]
  if fileEXT == 'xml':
   arrayName.append(item['name']) 
   #print (fileEXT)
    
for i in range(len(arrayName)): 
 print ('Name Index ' + str(i) + ': ' + arrayName[i])
 print ('Cat Number ' + str(i) + ': ' + children[i])
 print (xmlFileAndPathArray[i] + '\n')

Name Index 0: USA-Colorado-Silverton_Caldera-2018-AMTCC04.xml
Cat Number 0: 5ce32ef6e4b0f7ebfdfb8f2d
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG01\USA-Colorado-Silverton_Caldera-2018-AMTAG01.xml

Name Index 1: USA-Colorado-Silverton_Caldera-2018-AMTGK07.xml
Cat Number 1: 5ce42c04e4b0654fe0bce479
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG02\USA-Colorado-Silverton_Caldera-2018-AMTAG02.xml

Name Index 2: USA-Colorado-Silverton_Caldera-2018-AMTGK01.xml
Cat Number 2: 5ce42946e4b0654fe0bce448
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG03\USA-Colorado-Silverton_Caldera-2018-AMTAG03.xml

Name Index 3: USA-Colorado-Silverton_Caldera-2018-AMTGK05.xml
Cat Number 3: 5ce42b7ae4b0654fe0bce46f
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG04\USA-Colorado-Silverton_Caldera-2018-AMTAG04.xml

Name Index 4: USA-Colorado-Silverton_Caldera-2018-AMTMB02.xml
Cat Number 4: 5ce42d0ae4b0654fe0bce498
C:\CurrentWork\DataReleases\Silverton2019\Data\AMTAG05\USA-Colorado-Silverton_C

In [11]:
#Now we loop through our name array and if it matches use tha name index for that child replace
for i in range(len(children)):
 #print (arrayName[i])
 for j in range(len(xmlFileNameArray)):
  #print ('\t' + xmlFileNameArray[j])
  if xmlFileNameArray[j] == arrayName[i]:
    #here is were we would replace a value - !!!Uncomment below to activate!!!
    #childItem_json = sb.get_item(children[i])
    #sb.replace_file(xmlFileAndPathArray[j], childItem_json)
    print ('ScienceBase XML Replaced')   
    print ('\tFile Name:' + xmlFileNameArray[j] + '\n\tChild Name: ' + arrayName[i] + '\n\tCatalog Number: ' + children[i] + '\n\n')

ScienceBase XML Replaced
	File Name:USA-Colorado-Silverton_Caldera-2018-AMTCC03.xml
	Child Name: USA-Colorado-Silverton_Caldera-2018-AMTCC03.xml
	Catalog Number: 5ce32ebee4b0f7ebfdfb8f2b


